In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

In [ ]:
loc = "../docs/Python_codes/PartD.csv"

In [10]:
#upload data;select variables of interest

data_raw = pd.read_csv(loc,sep=',',encoding='cp1252')
dt = data_raw.loc[:,('regionid','year', 'enrol_tot_num','enrol_lis_num','pdp', 'Part_D_Total_Premium','basic','Part_D_Drug_Deductible','unrestricted_drug','in_area_flag','Parent_Organization','Part_D_Basic_Premium')]

#have a look at the structure of the following variables:
#dt.describe()     
dt[['regionid','year']].describe()

#or use groupby
dt.groupby(['regionid','year']).enrol_tot_num.sum()
dt.groupby(['regionid','year','pdp']).enrol_tot_num.sum()


#note here there is a problem with missing variables
dt['enrol_lis_num'].isna().sum()
dt=dt.loc[dt['enrol_lis_num'].isna()==0,:] #excluding all observations with missing values


#create dummies for Parent organization and region (need to delete one)
dummies_organization=pd.get_dummies(dt.loc[:,'Parent_Organization'] ).drop('XLHealth Corporation',axis=1) 
dummies_region=pd.get_dummies(dt.loc[:,'regionid'] ).drop(34,axis=1) 
dummies_year=pd.get_dummies(dt.loc[:,'year']).drop(2013,axis=1)   

dt= dt.join([dummies_organization,dummies_region,dummies_year])

#market shares

dt['mkt_id']= dt['regionid'].apply(str)+dt['year'].apply(str) #define market id region-year

dt['demand'] = dt.enrol_tot_num - dt.enrol_lis_num #calculate s_numerator (market share numerator for all j (j=0 included) in all R markets and add to data table

s0 = dt.loc[dt['pdp']==0].groupby('mkt_id').sum().demand #select "outside good" plans - demand for outside good plans
s = dt.groupby('mkt_id').sum().demand  #total demand in each market

mj= dt.loc[dt['pdp']==1].join(s, on='mkt_id',rsuffix='_total').join(s0,on='mkt_id',rsuffix='_0') #select plans pdp==1 (inside goods) add column with corresponding market total and outside good total
m= mj.assign(mkt_sh_j= lambda x: x['demand']/x['demand_total']).assign(mkt_sh_0= lambda x: x['demand_0']/x['demand_total'])

C:\Users\feder\AppData\Local\Temp\ipykernel_18000\933265762.py:3: DtypeWarning: Columns (19,76) have mixed types. Specify dtype option on import or set low_memory=False.
  data_raw = pd.read_csv(loc,sep=',',encoding='cp1252')


In [11]:
# ============================================================
# Logit OLS with FE and clustered SEs
# ============================================================

# dependent variable: log(s_j) − log(s_0)
y0 = np.log(m['mkt_sh_j']) - np.log(m['mkt_sh_0'])
y  = y0.mask(np.isinf(y0) | np.isnan(y0)).astype(float)
y.name = 'y'

# main regressors
X_main = m[['Part_D_Total_Premium',
            'basic',
            'Part_D_Drug_Deductible',
            'unrestricted_drug',
            'in_area_flag']]

# fixed effects (drop_first=True to avoid collinearity with constant)
org_dummies    = pd.get_dummies(m['Parent_Organization'], drop_first=True).astype(int)
region_dummies = pd.get_dummies(m['regionid'],           drop_first=True).astype(int)
year_dummies   = pd.get_dummies(m['year'],               drop_first=True).astype(int)

# design matrix: characteristics + FE
X_ols = pd.concat([X_main, org_dummies, region_dummies, year_dummies], axis=1)
X_ols = X_ols.astype(float)
X_ols = sm.add_constant(X_ols)

# build one regression dataframe and cluster by market
ols_df = pd.concat([y, X_ols, m['mkt_id']], axis=1).dropna()
y_reg  = ols_df['y']
X_reg  = ols_df.drop(columns=['y', 'mkt_id'])
groups = ols_df['mkt_id']   # cluster dimension: market = region×year

# OLS with cluster-robust SEs at the market level
model_ols = sm.OLS(y_reg, X_reg).fit(
    cov_type='cluster',
    cov_kwds={'groups': groups}
)

print(model_ols.summary())

c:\Users\feder\OneDrive - Università Commerciale Luigi Bocconi\Bocconi\Materiale aa 2025-26\IO\CA\Python_codes\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.669
Model:                            OLS   Adj. R-squared:                  0.664
Method:                 Least Squares   F-statistic:                     4899.
Date:                Fri, 19 Dec 2025   Prob (F-statistic):               0.00
Time:                        14:28:48   Log-Likelihood:                -14664.
No. Observations:                9180   AIC:                         2.959e+04
Df Residuals:                    9051   BIC:                         3.050e+04
Df Model:                         128                                         
Covariance Type:              cluster                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

c:\Users\feder\OneDrive - Università Commerciale Luigi Bocconi\Bocconi\Materiale aa 2025-26\IO\CA\Python_codes\.venv\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 134, but rank is 120
  warnings.warn('covariance of constraints does not have full '


In [12]:
# =============================================================================
# 2SLS – Hausman instrument (logit with IV)
# =============================================================================

# -------- 1. Construct Hausman instrument (same as before) --------
IV = dt.loc[dt['pdp'] == 1,
            ('Part_D_Total_Premium', 'Parent_Organization', 'basic',
             'regionid', 'year')]

# own-market totals (ParentOrg × basic × region × year)
temp1 = IV.groupby(['Parent_Organization', 'basic', 'regionid', 'year']).sum()
temp2 = IV.groupby(['Parent_Organization', 'basic', 'regionid', 'year']).count()
IV = IV.join(temp1, on=['Parent_Organization','basic','regionid','year'],
             rsuffix='_mktsum')
IV = IV.join(temp2, on=['Parent_Organization','basic','regionid','year'],
             rsuffix='_mktcount')

# totals over all regions (ParentOrg × basic × year)
temp3 = IV.groupby(['Parent_Organization', 'basic', 'year']).sum().Part_D_Total_Premium
temp4 = IV.groupby(['Parent_Organization', 'basic', 'year']).count().Part_D_Total_Premium
IV = IV.join(temp3, on=['Parent_Organization','basic','year'],
             rsuffix='_allsum')
IV = IV.join(temp4, on=['Parent_Organization','basic','year'],
             rsuffix='_allcount')

# average premium of "other" markets for same org × basic × year
ivhaus = (IV.Part_D_Total_Premium_allsum - IV.Part_D_Total_Premium_mktsum) \
         / (IV.Part_D_Total_Premium_allcount - IV.Part_D_Total_Premium_mktcount)
ivhaus.name = 'ivhaus'

# -------- 2. Build regression sample (inside goods + y + instrument) --------
m2 = m.join(ivhaus).join(y)
m2 = m2.dropna()

# -------- 3. Exogenous covariates: SAME as OLS, excluding premium --------
X_char = m2[['basic',
             'Part_D_Drug_Deductible',
             'unrestricted_drug',
             'in_area_flag']]

org_dummies    = pd.get_dummies(m2['Parent_Organization'], drop_first=True).astype(int)
region_dummies = pd.get_dummies(m2['regionid'],           drop_first=True).astype(int)
year_dummies   = pd.get_dummies(m2['year'],               drop_first=True).astype(int)

X_exog = pd.concat([X_char, org_dummies, region_dummies, year_dummies], axis=1).astype(float)

# -------- 4. First stage: premium on X_exog + ivhaus (clustered) --------
X_fs = sm.add_constant(pd.concat([X_exog, m2['ivhaus']], axis=1))
fs_model = sm.OLS(m2['Part_D_Total_Premium'], X_fs).fit(
    cov_type='cluster', cov_kwds={'groups': m2['mkt_id']}
)
print("\n=== First stage: Premium on X_exog + ivhaus ===")
print(fs_model.summary())

# Reduced form (optional, but useful for the write-up)
rf_model = sm.OLS(m2['y'], X_fs).fit(
    cov_type='cluster', cov_kwds={'groups': m2['mkt_id']}
)
print("\n=== Reduced form: y on X_exog + ivhaus ===")
print(rf_model.summary())

# -------- 5. Structural 2SLS via IV2SLS, then cluster SEs --------
# Structural regressors: const + premium + X_exog
X_struct = sm.add_constant(
    pd.concat([m2['Part_D_Total_Premium'], X_exog], axis=1)
).astype(float)

# Instruments: const + X_exog + ivhaus
Z = sm.add_constant(
    pd.concat([m2['ivhaus'], X_exog], axis=1)
).astype(float)

# Plain IV2SLS fit
iv_model_raw = IV2SLS(m2['y'], X_struct, Z).fit()

# Add cluster-robust covariance (cluster by market mkt_id)
iv_model = iv_model_raw.get_robustcov_results(
    cov_type='cluster',
    groups=m2['mkt_id']
)

print("\n=== 2SLS logit (Hausman IV, clustered by market) ===")
print(iv_model.summary())



=== First stage: Premium on X_exog + ivhaus ===
                             OLS Regression Results                             
Dep. Variable:     Part_D_Total_Premium   R-squared:                       0.693
Model:                              OLS   Adj. R-squared:                  0.690
Method:                   Least Squares   F-statistic:                     757.5
Date:                  Fri, 19 Dec 2025   Prob (F-statistic):          1.12e-276
Time:                          14:28:49   Log-Likelihood:                -34602.
No. Observations:                  8795   AIC:                         6.937e+04
Df Residuals:                      8714   BIC:                         6.994e+04
Df Model:                            80                                         
Covariance Type:                cluster                                         
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------

### Comment per i miei G
The issue with the nested logit is that the conditional share s_nest is also endogenous. The Hausman instrument is not enough because we now have two endogenous regressors (price and 
s_nest) but only one excluded instrument, and that instrument mainly shifts prices, not the within–nest share. For identification in 2SLS we need at least as many excluded instruments as endogenous variables, so we add a BLP-style instrument based on other plans’ characteristics (e.g. average rival deductibles) to instrument the nest term.

In [13]:
# =============================================================================
# Nested Logit
# =============================================================================

# -------- 1. Build nest shares and log(s_j | nest) --------
# two nests based on `basic` (0 = enhanced, 1 = basic)
total_nest = m.groupby(['regionid', 'year', 'basic'])['mkt_sh_j'].sum()

# attach nest total share to each product
m_nest = m.join(total_nest, on=['regionid', 'year', 'basic'], rsuffix='_nest')

# s_j|g = s_j / s_g, and log(s_j|g)
m_nest['s_nest'] = np.log(m_nest['mkt_sh_j'] / m_nest['mkt_sh_j_nest'])

# merge s_nest into the IV sample (m2 already has y, ivhaus, etc.)
m3 = m2.join(m_nest['s_nest'])
m3 = m3.dropna()

# -------- 2. Exogenous covariates: SAME structure as in logit IV --------
X_char_nl = m3[['basic',
                'Part_D_Drug_Deductible',
                'unrestricted_drug',
                'in_area_flag']]

org_dummies_nl    = pd.get_dummies(m3['Parent_Organization'], drop_first=True).astype(int)
region_dummies_nl = pd.get_dummies(m3['regionid'],           drop_first=True).astype(int)
year_dummies_nl   = pd.get_dummies(m3['year'],               drop_first=True).astype(int)

X_exog_nl = pd.concat(
    [X_char_nl, org_dummies_nl, region_dummies_nl, year_dummies_nl],
    axis=1
).astype(float)

# -------- 3. OLS nested logit (Berry eq. 28) with FE, clustered by market --------
X_nl_ols = sm.add_constant(
    pd.concat([m3['Part_D_Total_Premium'], X_exog_nl, m3['s_nest']], axis=1)
)

ols_nl_df = pd.concat([m3['y'], X_nl_ols, m3['mkt_id']], axis=1).dropna()
y_nl_ols  = ols_nl_df['y']
X_nl_ols  = ols_nl_df.drop(columns=['y', 'mkt_id'])
groups_nl = ols_nl_df['mkt_id']

model_nl_ols = sm.OLS(y_nl_ols, X_nl_ols).fit(
    cov_type='cluster',
    cov_kwds={'groups': groups_nl}
)

print("\n=== Nested logit OLS (with FE, clustered by market) ===")
print(model_nl_ols.summary())

# -------- 4. BLP-style instrument for s_nest (based on deductibles) --------
# build on dt, but only for inside goods
ivblp_df = dt.loc[dt['pdp'] == 1,
                  ('Part_D_Drug_Deductible', 'basic', 'regionid', 'year')]

# own-market totals: basic × region × year
temp1 = ivblp_df.groupby(['basic', 'regionid', 'year']).sum()
temp2 = ivblp_df.groupby(['basic', 'regionid', 'year']).count()
ivblp_df = ivblp_df.join(temp1, on=['basic', 'regionid', 'year'], rsuffix='_mktsum')
ivblp_df = ivblp_df.join(temp2, on=['basic', 'regionid', 'year'], rsuffix='_mktcount')

# totals over all regions: basic × year
temp3 = ivblp_df.groupby(['basic', 'year']).sum().Part_D_Drug_Deductible
temp4 = ivblp_df.groupby(['basic', 'year']).count().Part_D_Drug_Deductible
ivblp_df = ivblp_df.join(temp3, on=['basic', 'year'], rsuffix='_allsum')
ivblp_df = ivblp_df.join(temp4, on=['basic', 'year'], rsuffix='_allcount')

# average deductible of "other markets" for same basic × year (BLP-style)
ivblp = (ivblp_df.Part_D_Drug_Deductible_allsum
         - ivblp_df.Part_D_Drug_Deductible_mktsum) / \
        (ivblp_df.Part_D_Drug_Deductible_allcount
         - ivblp_df.Part_D_Drug_Deductible_mktcount)
ivblp.name = 'ivblp'

# attach ivblp to m3 (indices align: both are inside-good rows)
m3 = m3.join(ivblp)
m3 = m3.dropna()

# -------- 5. IV nested logit: premium and s_nest endogenous --------
# Structural regressors: const + premium + s_nest + exogenous covariates
X_struct_nl = sm.add_constant(
    pd.concat([m3['Part_D_Total_Premium'], m3['s_nest'], X_exog_nl.loc[m3.index]], axis=1)
).astype(float)

# Instruments: const + exogenous covariates + ivhaus (for price) + ivblp (for s_nest)
Z_nl = sm.add_constant(
    pd.concat([X_exog_nl.loc[m3.index], m3['ivhaus'], m3['ivblp']], axis=1)
).astype(float)

# raw IV2SLS fit
nl_iv_raw = IV2SLS(m3['y'], X_struct_nl, Z_nl).fit()

# cluster-robust covariance at market level
nl_iv = nl_iv_raw.get_robustcov_results(
    cov_type='cluster',
    groups=m3['mkt_id']
)

print("\n=== Nested logit 2SLS (premium & s_nest IV'd, clustered by market) ===")
print(nl_iv.summary())


c:\Users\feder\OneDrive - Università Commerciale Luigi Bocconi\Bocconi\Materiale aa 2025-26\IO\CA\Python_codes\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)



=== Nested logit OLS (with FE, clustered by market) ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.906
Method:                 Least Squares   F-statistic:                     7258.
Date:                Fri, 19 Dec 2025   Prob (F-statistic):               0.00
Time:                        14:28:49   Log-Likelihood:                -8412.5
No. Observations:                8795   AIC:                         1.699e+04
Df Residuals:                    8713   BIC:                         1.757e+04
Df Model:                          81                                         
Covariance Type:              cluster                                         
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------

In [14]:
# =============================================================================
# Nested Logit (OLS + 2SLS with Hausman + BLP instruments)
# =============================================================================

# -------- 1. Build nest shares and s_nest = log(s_j | nest) --------
# Nest defined by `basic` (0 = enhanced, 1 = basic)
total_nest = m.groupby(['regionid', 'year', 'basic'])['mkt_sh_j'].sum()

# attach nest total share to each product
m_nest = m.join(total_nest, on=['regionid', 'year', 'basic'], rsuffix='_nest')

# conditional share within nest and its log
m_nest['s_nest'] = np.log(m_nest['mkt_sh_j'] / m_nest['mkt_sh_j_nest'])

# -------- 2. BLP instrument: avg deductible of *other* plans in same nest & market --------
g = m.groupby(['basic', 'regionid', 'year'])
sum_d   = g['Part_D_Drug_Deductible'].transform('sum')
count_d = g['Part_D_Drug_Deductible'].transform('count')

iv_blp = (sum_d - m['Part_D_Drug_Deductible']) / (count_d - 1)
iv_blp = iv_blp.where(count_d > 1, np.nan)   # undefined if only one plan in nest
iv_blp.name = 'iv_blp'

# -------- 3. Build nested-logit sample: start from m2 (inside goods + y + ivhaus) --------
m3 = m2.join(m_nest['s_nest']).join(iv_blp)
m3 = m3.dropna()

# -------- 4. Exogenous covariates: SAME structure as logit IV --------
X_char_nl = m3[['basic',
                'Part_D_Drug_Deductible',
                'unrestricted_drug',
                'in_area_flag']]

org_dummies_nl    = pd.get_dummies(m3['Parent_Organization'], drop_first=True).astype(int)
region_dummies_nl = pd.get_dummies(m3['regionid'],           drop_first=True).astype(int)
year_dummies_nl   = pd.get_dummies(m3['year'],               drop_first=True).astype(int)

X_exog_nl = pd.concat(
    [X_char_nl, org_dummies_nl, region_dummies_nl, year_dummies_nl],
    axis=1
).astype(float)

# -------- 5. OLS nested logit with FE, clustered by market --------
# y = beta0 - alpha * price + beta' X + sigma * s_nest + xi

X_nl_ols = sm.add_constant(
    pd.concat([m3['Part_D_Total_Premium'], X_exog_nl, m3['s_nest']], axis=1)
).astype(float)

ols_nl_df = pd.concat([m3['y'], X_nl_ols, m3['mkt_id']], axis=1).dropna()
y_nl_ols  = ols_nl_df['y']
X_nl_ols  = ols_nl_df.drop(columns=['y', 'mkt_id'])
groups_nl = ols_nl_df['mkt_id']

model_nl_ols = sm.OLS(y_nl_ols, X_nl_ols).fit(
    cov_type='cluster',
    cov_kwds={'groups': groups_nl}
)

print("\n=== Nested logit OLS (with FE, clustered by market) ===")
print(model_nl_ols.summary())

# -------- 6. IV nested logit: price and s_nest endogenous --------
# Endogenous: Part_D_Total_Premium, s_nest
# Instruments: exogenous X + ivhaus (for price) + iv_blp (for s_nest)

# Structural regressors: const + price + s_nest + exogenous
X_struct_nl = sm.add_constant(
    pd.concat([m3['Part_D_Total_Premium'], m3['s_nest'], X_exog_nl], axis=1)
).astype(float)

# Instrument matrix: const + exogenous + ivhaus + iv_blp
Z_nl = sm.add_constant(
    pd.concat([X_exog_nl, m3['ivhaus'], m3['iv_blp']], axis=1)
).astype(float)

# -------- 6a. First stage for PRICE --------
fs_price = sm.OLS(m3['Part_D_Total_Premium'], Z_nl).fit(
    cov_type='cluster',
    cov_kwds={'groups': m3['mkt_id']}
)
print("\n=== First stage for price: premium on X_exog + ivhaus + iv_blp ===")
print(fs_price.summary())

# -------- 6b. First stage for S_NEST --------
fs_snest = sm.OLS(m3['s_nest'], Z_nl).fit(
    cov_type='cluster',
    cov_kwds={'groups': m3['mkt_id']}
)
print("\n=== First stage for s_nest: s_nest on X_exog + ivhaus + iv_blp ===")
print(fs_snest.summary())

# -------- 6c. 2SLS nested logit (IV2SLS) with clustered SEs --------
nl_iv_raw = IV2SLS(m3['y'], X_struct_nl, Z_nl).fit()

nl_iv = nl_iv_raw.get_robustcov_results(
    cov_type='cluster',
    groups=m3['mkt_id']
)

print("\n=== Nested logit 2SLS (price & s_nest IV'd, clustered by market) ===")
print(nl_iv.summary())

c:\Users\feder\OneDrive - Università Commerciale Luigi Bocconi\Bocconi\Materiale aa 2025-26\IO\CA\Python_codes\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)



=== Nested logit OLS (with FE, clustered by market) ===
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.911
Method:                 Least Squares   F-statistic:                     9175.
Date:                Fri, 19 Dec 2025   Prob (F-statistic):               0.00
Time:                        14:28:50   Log-Likelihood:                -8139.8
No. Observations:                8783   AIC:                         1.644e+04
Df Residuals:                    8701   BIC:                         1.702e+04
Df Model:                          81                                         
Covariance Type:              cluster                                         
                                                      coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------